This notebook contains testing of the Deepseek 7B Instruct quantized model on the MBPP dataset using a system prompt. Zero examples are provided with each task from the dataset (zero-shot).

# Installations and imports

In [ ]:
!pip install accelerate
!pip install bitsandbytes

In [ ]:
from tqdm import tqdm

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
import re
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Choosing dataset

## MBPP

In [ ]:
dataset = load_dataset("mbpp")
dataset

Generating train split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 374
    })
    test: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 90
    })
    prompt: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 10
    })
})

In [ ]:
dataset_prompt = dataset['prompt']

In [ ]:
dataset = dataset['test']

# Choosing model

## DeepSeek 6.7B Instruct

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = "deepseek-ai/deepseek-coder-6.7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map='auto', load_in_8bit=True)

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

## Codellama 7B Instruct

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = "codellama/CodeLlama-7b-Instruct-hf"
# #model = "deepseek-ai/deepseek-coder-1.3b-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model)
# model = AutoModelForCausalLM.from_pretrained(model, torch_dtype=torch.float16, device_map='auto', load_in_8bit=True)

# Generation and data preparation functions

In [ ]:
def make_prompt_list(dataset, shots, num_tests):
    prompt_list = []
    pattern = r'(?<=assert\s)\w+\s*\('
    for i in range(num_tests):
        s = dataset['test_list'][i][0]
        func_name = re.search(pattern, s)
        func_name = func_name.group().strip(' (') if func_name else ''
        prompt = dataset['text'][i] + ' The function should have the following name: ' + func_name + '.\n'
        if shots > 0:
            prompt += 'The code should also pass these tests: '
            for j in range(shots):
                prompt +=  dataset['test_list'][i][j] + ', '
            prompt = prompt[:len(prompt) - 2]
        prompt_list.append(prompt)
    return prompt_list


def generate(model, dataset, shots=0, num_tests=len(dataset), do_sample=False, top_p=-1.0, top_k=0, temperature=1.0, dialog=[]):
    prompt_list = make_prompt_list(dataset, shots, num_tests)
    model.eval()
    responses = []

    for prompt in prompt_list:
        messages = [
            {
                "role": "System",
                "content":
                    "You are a smart assistant in writing code that helps the user solve his tasks. "
                    "Below is an instruction describing the task. Write an answer that exactly fulfills the user's request. "
            }
        ]

        messages.append(
            {
                "role": "User",
                "content": prompt
            }
        )

        print(messages)
        inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(input_ids=inputs, max_new_tokens=200, num_return_sequences=1, do_sample=do_sample, top_p=top_p, top_k=top_k, temperature=temperature, eos_token_id=tokenizer.eos_token_id)
            response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

        responses.append(response)

    return prompt_list, responses

# Testing

In [ ]:
def extract_code(text):
    code = []
    inside_function = False

    for line in text.split('\n'):
        if line.startswith('```python'):
            inside_function = True
        elif (line.startswith('import') or line.startswith('from') or line.startswith('def')) and inside_function == False:
            inside_function = True
            code.append(line)
        elif inside_function:
            if line == '```' or line.startswith('assert') or line.startswith('# Test'):
                return '\n'.join(code)
            else:
                code.append(line)
    return '\n'.join(code)

In [ ]:
def test(num_tests, dataset, responses):
    score = 0
    codes = []
    tests = []
    results = []
    for i in range(num_tests):
        code = extract_code(responses[i])
        codes.append(code)
        test = dataset['test_list'][i][0] + '\n' + dataset['test_list'][i][1] + '\n' + dataset['test_list'][i][2]
        tests.append(test)
        code = code + '\n' + test if code else test
        code = code.strip()
        flag = True
        try:
            exec(code)
        except (AssertionError, TypeError, IndentationError, NameError, SyntaxError, AttributeError, EOFError, ImportError, IndexError, KeyError, ValueError, ZeroDivisionError):
            flag = False
            pass
        else:
            score += 1

        results.append('Ok' if flag else 'Error')
    return codes, tests, results

In [ ]:
def save_results(df, parameters, prompt_list, responses, codes, tests, results):
    bias = len(df)
    df.loc[bias, 'parameters'] = parameters
    for i in range(len(prompt_list)):
        df.loc[bias + i, 'prompt'] = prompt_list[i]
        df.loc[bias + i, 'response'] = responses[i]
        df.loc[bias + i, 'code'] = codes[i]
        df.loc[bias + i, 'tests'] = tests[i]
        df.loc[bias + i, 'result'] = results[i]

In [ ]:
def predict(df, model, dataset, shots=0, num_tests=100, do_sample=False, top_p=-1.0, top_k=0, temperature=1.0, dialog=[]):
    prompt_list, responses = generate(model=model, dataset=dataset, shots=shots, num_tests=num_tests, do_sample=False, top_p=top_p, top_k=top_k, temperature=temperature, dialog=dialog)
    codes, tests, results = test(dataset=dataset, num_tests=num_tests, responses=responses)
    parameters = str({'shots': shots, 'do_sample': do_sample, 'top_p': top_p, 'top_k': top_k, 'temperature': temperature})
    save_results(df, parameters, prompt_list, responses, codes, tests, results)

# Getting results

In [ ]:
NUM_TESTS = len(dataset)

In [ ]:
NUM_TESTS

500

In [ ]:
df = pd.DataFrame(columns = ['parameters', 'prompt', 'response', 'code', 'tests', 'result'])
predict(df, model=model, dataset=dataset, shots=0, num_tests=NUM_TESTS, do_sample=True, top_k=40, top_p=0.95, temperature=0.1, dialog=[])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


[{'role': 'System', 'content': "You are a smart assistant in writing code that helps the user solve his tasks. Below is an instruction describing the task. Write an answer that exactly fulfills the user's request. "}, {'role': 'User', 'content': 'Write a python function to remove first and last occurrence of a given character from the string. The function should have the following name: remove_Occ.\n'}]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


[{'role': 'System', 'content': "You are a smart assistant in writing code that helps the user solve his tasks. Below is an instruction describing the task. Write an answer that exactly fulfills the user's request. "}, {'role': 'User', 'content': 'Write a function to sort a given matrix in ascending order according to the sum of its rows. The function should have the following name: sort_matrix.\n'}]


# Saving results

In [ ]:
df.to_csv('deepseek_mbpp_short_chat_zero_shot.csv', index=False)